<a href="https://colab.research.google.com/github/RogerJL/LTU/blob/main/eMaintenance/machine_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2: Machine data

In [1]:
%matplotlib notebook

from math import sqrt

import numpy as np
import pandas as pd

import matplotlib
import lets_plot as lplt
import matplotlib.pyplot as plt
#matplotlib.use("TkAgg")

from scipy.stats import weibull_min, norm, uniform, expon


In [2]:
# read the data file into a dataframe
df = pd.read_csv('machine_data.csv')


# Data re-conditioning 
Drop the index
Rename column and make manufacturer names consistent

In [3]:
df.drop(columns='Unnamed: 0', inplace=True)
df.loc[df.manufacturef == 'c', 'manufacturef'] = 'C'
df.loc[:, 'manufacturer'] = df.manufacturef
df.drop(columns='manufacturef', inplace=True)

# Group data by manufacturer
## Tabular data


In [4]:
bounds_df = df.groupby('manufacturer').agg(  # describe()
    {
        "time": ["min", "max", "median", "mean"],
        "load": ["min", "max", "median", "mean", "var"],
    }
)
print(bounds_df.T)
print("     mode     ", end="")
for _name, dfa in df.groupby('manufacturer'):
    load = dfa[['load']]
    print(f"{' & '.join(map(lambda v: str(int(v[0])), load.round().mode().values)):11s}", end='')

manufacturer          A          B          C
time min      15.276417  10.881338   4.519349
     max      44.283424  35.422287  24.296367
     median   25.350244  18.816761  12.305028
     mean     25.685249  19.230276  12.696966
load min      67.770206  68.364413  68.892794
     max      80.541808  80.167333  82.348464
     median   74.464071  74.690345  74.335344
     mean     74.497625  74.686092  74.376665
     var       4.609115   4.238006   4.699577
     mode     74         75         73 & 74    

Ranges in time and load for the manufacturers are

|   | time         | load         |
|---|--------------|--------------|
| A | 15.3 .. 44.3 | 66.8 .. 80.5 |
| B | 10.9 .. 35.4 | 68.4 .. 80.2 |
| C |  4.5 .. 24.3 | 68.9 .. 82.3 |

The most common load for the different manufacturers are
A: 74
B: 75
C: 73 and 74

# Prepare for plotting

In [5]:
lplt.LetsPlot.setup_html()

# Plot boxplot for each manufacturer
this shows percentiles including outliers

In [6]:

man_box = (lplt.ggplot(df)
           + lplt.geom_boxplot(lplt.aes(x='manufacturer', y='time',
                                        color='manufacturer', fill='manufacturer'),
                               outlier_shape=21, outlier_size=1.5, size=2,
                               alpha=.5, width=.5, show_legend=False)
           )
lplt.ggsave(man_box, "load_manufacturer_boxplot.svg")
man_box.show()

It is easy to see that there is an actual difference in time between the manufacturers as the median line is outside the others inter quartile range.
The numbers of outliers are worrying, is the distriubution not a normal distribution? This will be answered below.

# Plot relation between load and time

In [7]:
from math import floor, ceil
plot_minimum = df.min(numeric_only=True).apply(lambda x: floor(x - 0.5))
plot_maximum = df.max(numeric_only=True).apply(lambda x: ceil(x + 0.5))

grpByManu = df.groupby(['manufacturer'], sort=True)

relation = (lplt.ggplot(df)
 + lplt.geom_point(lplt.aes(x='load', y='time', color='manufacturer'))
 + lplt.ggtitle("Relation between load and time")
 )

relation.show()
lplt.ggsave(relation, "manufacturer_relation.svg")


'/home/roger/jupyter_data/gits/LTU/eMaintenance/lets-plot-images/manufacturer_relation.svg'

## Can a function be made to fit?

In [8]:
from scipy.optimize import curve_fit

def generic_fn(x, a, b, c, d):
    return (a * x + b) / (c * x + d)

#fit_relation = lplt.ggplot()
fit_relation = relation
plot_range = np.linspace(plot_minimum.load, plot_maximum.load, 20)
for name, dfa in grpByManu:    
    popt, pcov = curve_fit(generic_fn, dfa.load, dfa.time)
    fit_relation = fit_relation + lplt.geom_line(lplt.aes(x=plot_range, y=generic_fn(plot_range, *popt)))
    print(f"{name[0]}: time = ({popt[0]} * load + {popt[1]}) / ({popt[2]} * load + {popt[3]})")
    
fit_relation.show()

A: time = (-10.404977883385328 * load + 1020.2457737472195) / (0.3903253005577119 * load + -19.386875708168287)
B: time = (-5097.391796136691 * load + 479329.28963440476) / (248.3635206641907 * load + -13314.36171268699)
C: time = (-50.840518481987296 * load + 4573.3684158225) / (3.8103327433212337 * load + -218.79811430284724)


Not that bad fit. But parameters run wild. Would not extrapolate from these...

# Histograms with matching distributions

I would assume load to have normal distribution, as it is something physical sampled
Other distributions can have various skewnesses and kurtosis built in
If it is not a normal distribution I would question measurements!

6sigma is a methodology for improving manufacturing by reducing variations, the main idea is to have 6sigma distance between a specification limit and process mean output.

Time to failure fits best with Weibull distribution, not suprising as it handles breakage.

In [9]:
fig, axs = plt.subplots(3, 2, sharex=False, sharey=False)
for ax in axs[:, 0]:
    ax.set(xlim=(plot_minimum['load'], plot_maximum['load']))
for ax in axs[:, 1]:
    ax.set(xlim=(plot_minimum['time'], plot_maximum['time']))

for index, (name, dfa) in enumerate(grpByManu):
    name = "Manufacturer " + name[0]

    load = dfa['load']
    time = dfa['time']

    #%%
    bins = 20
    n, bins, patches = axs[index, 0].hist(load, bins=bins, label=name)
    axs[index,0].title.set_text(f"Histogram of load distribution")
    axs[index, 0].legend()
    load_mean, load_std = norm.fit(load)

    def load_cdf(range):
        return load.size * norm.cdf(range, load_mean, load_std)
    
    def plot_cdf(ax, range, cdf):
        left = range[:-1]
        right = range[1:]
        ax.plot((left + right)/2, (cdf(right) - cdf(left)))

    plot_cdf(axs[index, 0], bins, load_cdf)

    #%%
    bins = 20
    n, bins, patches = axs[index, 1].hist(time, bins=bins, label=name)
    axs[index, 1].title.set_text(f"Histogram of time distribution")
    axs[index, 1].legend()
    
    c, loc, scale = weibull_min.fit(time)
    
    def time_cdf(range):
        return time.size * weibull_min.cdf(range, c=c, loc=loc, scale=scale)
    
    plot_cdf(axs[index, 1], bins, time_cdf)
    
plt.show()

<IPython.core.display.Javascript object>

In [10]:

plt.savefig("manufacturers_hist_fit.svg", dpi=150)

# Conclusion
Manufacturer A has the best performance.

There is almost no difference in load while the difference in time to breakage is quite big.
Manufacturer B could be an alternative if much cheaper (inluding cost to replace).

# Appendix: Other plots

## Plot a histogram with all data

In [11]:
man_hist = lplt.ggplot(df, lplt.aes(x='load', fill='manufacturer'))
man_hist = (man_hist
 + lplt.ggsize(700, 300)
 + lplt.scale_fill_brewer(type='seq')
 + lplt.geom_histogram(position='dodge', alpha=0.7)
 + lplt.theme(panel_grid_major_x='blank')
 + lplt.geom_vline(lplt.aes(xintercept=bounds_df.load['mean'],
                            color=bounds_df.index.values),
                   linetype="dashed")
)
lplt.ggsave(man_hist, "load_manufacturer_histogram.svg")
man_hist.show()